In [2]:
!pip install -r requirements.txt

In [3]:
import polars as pl
import math

pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_width_chars(-1)

polars.config.Config

# Time variables

In [4]:
def read_dataset(location):
    read_settings = {
        "raise_if_empty": True,
        "schema_overrides": {"Time": pl.Time}
    }
    datetime = pl.datetime(
        year=pl.col("Date").dt.year(),
        month=pl.col("Date").dt.month(),
        day=pl.col("Date").dt.day(),
        hour=pl.col("Time").dt.hour(),
        minute=pl.col("Time").dt.minute(),
        second=pl.col("Time").dt.second()
    )
    return pl.read_excel(location, **read_settings).with_columns(
        pl.all().exclude("DateTime", "Date", "Time").replace(-200, None),
        DateTime=datetime,
    )

In [6]:
df = read_dataset("AirQualityUCI.xlsx").select("DateTime", "Date", "Time").hstack(pl.read_ipc("Model A1 denormalised.parquet"))

df.write_ipc("denormalised, no lags.parquet")
df.tail()

DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
datetime[μs],date,time,f64,f64,f64,bool,bool,bool,bool,bool,bool,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2005-04-04 10:00:00,2005-04-04,10:00:00,1.066871,0.999305,-0.037289,true,false,false,false,false,false,false,true,0.365341,0.930874,0.5,-0.866025,3.1,1314.25,694.291037,13.529605,1101.25,471.7,538.5,189.8,1374.25,1728.5,21.85,29.25,0.756824
2005-04-04 11:00:00,2005-04-04,11:00:00,1.066985,0.999278,-0.038006,true,false,false,false,false,false,false,true,0.399892,0.916562,0.258819,-0.965926,2.4,1162.5,484.50417,11.355157,1027.0,353.3,603.75,179.2,1263.5,1269.0,24.325,23.725,0.711864
2005-04-04 12:00:00,2005-04-04,12:00:00,1.067099,0.99925,-0.038722,true,false,false,false,false,false,false,true,0.433884,0.900969,1.2246e-16,-1.0,2.4,1142.0,469.482057,12.374538,1062.5,293.0,603.25,174.7,1240.75,1092.0,26.9,18.35,0.640649
2005-04-04 13:00:00,2005-04-04,13:00:00,1.067213,0.999222,-0.039439,true,false,false,false,false,false,false,true,0.467269,0.884115,-0.258819,-0.965926,2.1,1002.5,338.026969,9.547187,960.5,234.5,701.5,155.7,1041.0,769.75,28.325,13.55,0.513866
2005-04-04 14:00:00,2005-04-04,14:00:00,1.067327,0.999193,-0.040156,true,false,false,false,false,false,false,true,0.5,0.866025,-0.5,-0.866025,2.2,1070.75,574.001469,11.93206,1047.25,265.2,654.0,167.7,1128.5,816.0,28.5,13.125,0.502804


In [8]:
df2 = read_dataset("AirQualityUCI.xlsx").select("DateTime", "Date", "Time").hstack(pl.read_ipc("Model A1 full imputed dataset.parquet"))

df2.write_ipc("Z-score normalised, no lags.parquet")
df2.tail()

DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
datetime[μs],date,time,f64,f64,f64,bool,bool,bool,bool,bool,bool,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2005-04-04 10:00:00,2005-04-04,10:00:00,1.066871,0.999305,-0.037289,true,false,false,false,false,false,false,true,0.365341,0.930874,0.5,-0.866025,0.939983,1.080339,1.694102,0.630752,0.625113,1.683138,-1.53346,2.180491,-0.776192,1.773978,-0.014506,-1.019716,-1.285018
2005-04-04 11:00:00,2005-04-04,11:00:00,1.066985,0.999278,-0.038006,true,false,false,false,false,false,false,true,0.399892,0.916562,0.258819,-0.965926,0.511876,0.486384,1.284302,0.370274,0.363909,1.294739,-1.12691,1.966609,-1.237925,0.839558,0.31293,-1.377923,-1.43512
2005-04-04 12:00:00,2005-04-04,12:00:00,1.067099,0.99925,-0.038722,true,false,false,false,false,false,false,true,0.433884,0.900969,1.2246e-16,-1.0,0.511876,0.396262,1.248918,0.496827,0.490661,1.048023,-1.129858,1.874412,-1.33687,0.413334,0.639501,-1.739979,-1.678652
2005-04-04 13:00:00,2005-04-04,13:00:00,1.067213,0.999222,-0.039439,true,false,false,false,false,false,false,true,0.467269,0.884115,-0.258819,-0.965926,0.294707,-0.296353,0.884549,0.122382,0.116469,0.75925,-0.59247,1.475094,-2.276488,-0.512981,0.814731,-2.07831,-2.131281
2005-04-04 14:00:00,2005-04-04,14:00:00,1.067327,0.999193,-0.040156,true,false,false,false,false,false,false,true,0.5,0.866025,-0.5,-0.866025,0.369797,0.061097,1.476088,0.442929,0.436643,0.91812,-0.842324,1.729242,-1.848108,-0.364578,0.835999,-2.109121,-2.172022


# 01 scaled data

In [9]:
measured_values = [
    "CO(GT)",
    "PT08.S1(CO)",
    "NMHC(GT)",
    "C6H6(GT)",
    "PT08.S2(NMHC)",
    "NOx(GT)",
    "PT08.S3(NOx)",
    "NO2(GT)",
    "PT08.S4(NO2)",
    "PT08.S5(O3)",
    "T",
    "RH",
    "AH",
]

In [10]:
time_vars = list(df.schema)[:18]
time_vars

['DateTime',
 'Date',
 'Time',
 'linear_dt',
 'sin_time_of_year',
 'cos_time_of_year',
 'is_mon',
 'is_tue',
 'is_wed',
 'is_thu',
 'is_fri',
 'is_sat',
 'is_sun',
 'is_weekday',
 'sin_time_of_week',
 'cos_time_of_week',
 'sin_hour_of_day',
 'cos_hour_of_day']

In [11]:
df.slice(0, 6043).min().select(measured_values)

CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.041878,647.25,7.0,0.212798,396.5,2.0,325.25,2.0,779.0,261.0,3.175,9.175,0.375444


In [12]:
mins = df.slice(0, 6043).min().select(measured_values).transpose().to_series().to_list()
maxes = df.slice(0, 6043).max().select(measured_values).transpose().to_series().to_list()

In [13]:
df3_01_normed = df.select(
    *time_vars,
    *(((pl.col(value) - min) / (max - min)).alias(value) for value, min, max in zip(measured_values, mins, maxes))
)
df3_01_normed.write_ipc("min-max scaled, no lags.parquet")
df3_01_normed.tail()

DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
datetime[μs],date,time,f64,f64,f64,bool,bool,bool,bool,bool,bool,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2005-04-04 10:00:00,2005-04-04,10:00:00,1.066871,0.999305,-0.037289,true,false,false,false,false,false,false,true,0.365341,0.930874,0.5,-0.866025,0.323333,0.478995,0.19001,0.256877,0.437665,0.351446,0.118736,0.726384,0.298221,0.649839,0.450815,0.257372,0.20553
2005-04-04 11:00:00,2005-04-04,11:00:00,1.066985,0.999278,-0.038006,true,false,false,false,false,false,false,true,0.399892,0.916562,0.258819,-0.965926,0.249322,0.370018,0.132012,0.214932,0.391554,0.262855,0.155067,0.685385,0.242735,0.446363,0.510561,0.186538,0.1813
2005-04-04 12:00:00,2005-04-04,12:00:00,1.067099,0.99925,-0.038722,true,false,false,false,false,false,false,true,0.433884,0.900969,1.2246e-16,-1.0,0.249322,0.355296,0.127859,0.234596,0.4136,0.217736,0.154788,0.667979,0.231338,0.367984,0.572722,0.117628,0.142922
2005-04-04 13:00:00,2005-04-04,13:00:00,1.067213,0.999222,-0.039439,true,false,false,false,false,false,false,true,0.467269,0.884115,-0.258819,-0.965926,0.217604,0.255117,0.091516,0.180057,0.350256,0.173964,0.209493,0.59449,0.131263,0.225285,0.607121,0.05609,0.074597
2005-04-04 14:00:00,2005-04-04,14:00:00,1.067327,0.999193,-0.040156,true,false,false,false,false,false,false,true,0.5,0.866025,-0.5,-0.866025,0.228177,0.304129,0.156754,0.22606,0.40413,0.196935,0.183046,0.640904,0.1751,0.245766,0.611346,0.050641,0.068635


# yeo_johnson_scaled_measurements_training_data

# denormalized_imputed_training_data

# Lookback windows, denormalised

In [14]:
def fmt_wdw_num(num):
    if num == 0:
        return "(0hour)"
    
    elif num >=24*7 and num % (24*7) == 0:
        return f"-({num // (24*7)}week)"
    elif num + 1 >= 24*7 and (num + 1) % (24*7) == 0:
        return f"-({(num + 1) // (24*7)}week-1hour)"
    
    elif num >= 24 and num % 24 == 0:
        return f"-({num // 24}day)"
    elif num + 1 >= 24 and (num + 1) % 24 == 0:
        return f"-({(num + 1) // 24}day-1hour)"
    
    elif num < 24:
        return f"-({num}hour)"
    
    else:
        raise ValueError("Unsupported window size")

windows = [
    (1, 0),
    (2, 1),
    (3, 2),

    (2, 0),
    (3, 0),

    (24*7, 0),

    *((24*i, 24*i-1) for i in range(1, 7)), # This time yesterday, day before yesterday, ..., a week ago
    *((24*7*i, 24*7*i-1) for i in range(1, 3+1)), # This time last week, two weeks ago, three weeks ago
]

df3 = (
    df.select(
        *time_vars,
        *measured_values,
        *(
            (pl.col(var).rolling_sum(window_size=size_before_excl+1) - pl.col(var).rolling_sum(window_size=exclusion+1)).alias(f"{var}_wdw[{fmt_wdw_num(size_before_excl)}:{fmt_wdw_num(exclusion)}]") / (size_before_excl - exclusion)
            for size_before_excl, exclusion in windows
            for var in measured_values
        )
    )
    .with_columns(
        ((pl.col(f"{var}_wdw[-(1day):-(1day-1hour)]")        + pl.col(f"{var}_wdw[-(2day):-(2day-1hour)]"))/2).alias(f"{var}_wdw[-(2day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((2*pl.col(f"{var}_wdw[-(2day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(3day):-(3day-1hour)]"))/3).alias(f"{var}_wdw[-(3day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((3*pl.col(f"{var}_wdw[-(3day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(4day):-(4day-1hour)]"))/4).alias(f"{var}_wdw[-(4day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((4*pl.col(f"{var}_wdw[-(4day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(5day):-(5day-1hour)]"))/5).alias(f"{var}_wdw[-(5day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((5*pl.col(f"{var}_wdw[-(5day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(6day):-(6day-1hour)]"))/6).alias(f"{var}_wdw[-(6day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((6*pl.col(f"{var}_wdw[-(6day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]"))/7).alias(f"{var}_wdw[-(1week):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        *(
            ((pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]") + pl.col(f"{var}_wdw[-(2week):-(2week-1hour)]"))/2).alias(f"{var}_wdw[-(2week):-(1week-1hour):1week]")
            for var in measured_values
        ),
        *(
            ((pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]") + pl.col(f"{var}_wdw[-(2week):-(2week-1hour)]") + pl.col(f"{var}_wdw[-(3week):-(3week-1hour)]"))/3).alias(f"{var}_wdw[-(3week):-(1week-1hour):1week]")
            for var in measured_values
        )
    )
)

df3.head()

DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,CO(GT)_wdw[-(1hour):(0hour)],PT08.S1(CO)_wdw[-(1hour):(0hour)],NMHC(GT)_wdw[-(1hour):(0hour)],C6H6(GT)_wdw[-(1hour):(0hour)],PT08.S2(NMHC)_wdw[-(1hour):(0hour)],NOx(GT)_wdw[-(1hour):(0hour)],PT08.S3(NOx)_wdw[-(1hour):(0hour)],NO2(GT)_wdw[-(1hour):(0hour)],PT08.S4(NO2)_wdw[-(1hour):(0hour)],PT08.S5(O3)_wdw[-(1hour):(0hour)],T_wdw[-(1hour):(0hour)],RH_wdw[-(1hour):(0hour)],AH_wdw[-(1hour):(0hour)],CO(GT)_wdw[-(2hour):-(1hour)],PT08.S1(CO)_wdw[-(2hour):-(1hour)],NMHC(GT)_wdw[-(2hour):-(1hour)],C6H6(GT)_wdw[-(2hour):-(1hour)],PT08.S2(NMHC)_wdw[-(2hour):-(1hour)],NOx(GT)_wdw[-(2hour):-(1hour)],PT08.S3(NOx)_wdw[-(2hour):-(1hour)],NO2(GT)_wdw[-(2hour):-(1hour)],PT08.S4(NO2)_wdw[-(2hour):-(1hour)],PT08.S5(O3)_wdw[-(2hour):-(1hour)],T_wdw[-(2hour):-(1hour)],RH_wdw[-(2hour):-(1hour)],AH_wdw[-(2hour):-(1hour)],CO(GT)_wdw[-(3hour):-(2hour)],PT08.S1(CO)_wdw[-(3hour):-(2hour)],NMHC(GT)_wdw[-(3hour):-(2hour)],C6H6(GT)_wdw[-(3hour):-(2hour)],PT08.S2(NMHC)_wdw[-(3hour):-(2hour)],NOx(GT)_wdw[-(3hour):-(2hour)],PT08.S3(NOx)_wdw[-(3hour):-(2hour)],NO2(GT)_wdw[-(3hour):-(2hour)],PT08.S4(NO2)_wdw[-(3hour):-(2hour)],PT08.S5(O3)_wdw[-(3hour):-(2hour)],T_wdw[-(3hour):-(2hour)],RH_wdw[-(3hour):-(2hour)],AH_wdw[-(3hour):-(2hour)],CO(GT)_wdw[-(2hour):(0hour)],PT08.S1(CO)_wdw[-(2hour):(0hour)],NMHC(GT)_wdw[-(2hour):(0hour)],C6H6(GT)_wdw[-(2hour):(0hour)],PT08.S2(NMHC)_wdw[-(2hour):(0hour)],NOx(GT)_wdw[-(2hour):(0hour)],PT08.S3(NOx)_wdw[-(2hour):(0hour)],NO2(GT)_wdw[-(2hour):(0hour)],PT08.S4(NO2)_wdw[-(2hour):(0hour)],PT08.S5(O3)_wdw[-(2hour):(0hour)],T_wdw[-(2hour):(0hour)],RH_wdw[-(2hour):(0hour)],AH_wdw[-(2hour):(0hour)],CO(GT)_wdw[-(3hour):(0hour)],PT08.S1(CO)_wdw[-(3hour):(0hour)],NMHC(GT)_wdw[-(3hour):(0hour)],C6H6(GT)_wdw[-(3hour):(0hour)],PT08.S2(NMHC)_wdw[-(3hour):(0hour)],NOx(GT)_wdw[-(3hour):(0hour)],PT08.S3(NOx)_wdw[-(3hour):(0hour)],NO2(GT)_wdw[-(3hour):(0hour)],PT08.S4(NO2)_wdw[-(3hour):(0hour)],PT08.S5(O3)_wdw[-(3hour):(0hour)],T_wdw[-(3hour):(0hour)],RH_wdw[-(3hour):(0hour)],AH_wdw[-(3hour):(0hour)],CO(GT)_wdw[-(1week):(0hour)],PT08.S1(CO)_wdw[-(1week):(0hour)],NMHC(GT)_wdw[-(1week):(0hour)],C6H6(GT)_wdw[-(1week):(0hour)],PT08.S2(NMHC)_wdw[-(1week):(0hour)],NOx(GT)_wdw[-(1week):(0hour)],PT08.S3(NOx)_wdw[-(1week):(0hour)],NO2(GT)_wdw[-(1week):(0hour)],PT08.S4(NO2)_wdw[-(1week):(0hour)],PT08.S5(O3)_wdw[-(1week):(0hour)],T_wdw[-(1week):(0hour)],RH_wdw[-(1week):(0hour)],AH_wdw[-(1week):(0hour)],CO(GT)_wdw[-(1day):-(1day-1hour)],PT08.S1(CO)_wdw[-(1day):-(1day-1hour)],NMHC(GT)_wdw[-(1day):-(1day-1hour)],C6H6(GT)_wdw[-(1day):-(1day-1hour)],PT08.S2(NMHC)_wdw[-(1day):-(1day-1hour)],NOx(GT)_wdw[-(1day):-(1day-1hour)],PT08.S3(NOx)_wdw[-(1day):-(1day-1hour)],NO2(GT)_wdw[-(1day):-(1day-1hour)],PT08.S4(NO2)_wdw[-(1day):-(1day-1hour)],PT08.S5(O3)_wdw[-(1day):-(1day-1hour)],T_wdw[-(1day):-(1day-1hour)],RH_wdw[-(1day):-(1day-1hour)],AH_wdw[-(1day):-(1day-1hour)],CO(GT)_wdw[-(2day):-(2day-1hour)],PT08.S1(CO)_wdw[-(2day):-(2day-1hour)],NMHC(GT)_wdw[-(2day):-(2day-1hour)],C6H6(GT)_wdw[-(2day):-(2day-1hour)],PT08.S2(NMHC)_wdw[-(2day):-(2day-1hour)],NOx(GT)_wdw[-(2day):-(2day-1hour)],PT08.S3(NOx)_wdw[-(2day):-(2day-1hour)],NO2(GT)_wdw[-(2day):-(2day-1hour)],PT08.S4(NO2)_wdw[-(2day):-(2day-1hour)],PT08.S5(O3)_wdw[-(2day):-(2day-1hour)],T_wdw[-(2day):-(2day-1hour)],RH_wdw[-(2day):-(2day-1hour)],AH_wdw[-(2day):-(2day-1hour)],CO(GT)_wdw[-(3day):-(3day-1hour)],PT08.S1(CO)_wdw[-(3day):-(3day-1hour)],NMHC(GT)_wdw[-(3day):-(3day-1hour)],C6H6(GT)_wdw[-(3day):-(3day-1hour)],PT08.S2(NMHC)_wdw[-(3day):-(3day-1hour)],NOx(GT)_wdw[-(3day):-(3day-1hour)],PT08.S3(NOx)_wdw[-(3day):-(3day-1hour)],NO2(GT)_wdw[-(3day):-(3day-1hour)],PT08.S4(NO2)_wdw[-(3day):-(3day-1hour)],PT08.S5(O3)_wd

In [15]:
df3.write_ipc("denormalised, with lags.parquet")

# Lookback windows, Z-score normalised

In [16]:
df3_zscored = (
    df2.select(
        *time_vars,
        *measured_values,
        *(
            (pl.col(var).rolling_sum(window_size=size_before_excl+1) - pl.col(var).rolling_sum(window_size=exclusion+1)).alias(f"{var}_wdw[{fmt_wdw_num(size_before_excl)}:{fmt_wdw_num(exclusion)}]") / (size_before_excl - exclusion)
            for size_before_excl, exclusion in windows
            for var in measured_values
        )
    )
    .with_columns(
        ((pl.col(f"{var}_wdw[-(1day):-(1day-1hour)]")        + pl.col(f"{var}_wdw[-(2day):-(2day-1hour)]"))/2).alias(f"{var}_wdw[-(2day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((2*pl.col(f"{var}_wdw[-(2day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(3day):-(3day-1hour)]"))/3).alias(f"{var}_wdw[-(3day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((3*pl.col(f"{var}_wdw[-(3day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(4day):-(4day-1hour)]"))/4).alias(f"{var}_wdw[-(4day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((4*pl.col(f"{var}_wdw[-(4day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(5day):-(5day-1hour)]"))/5).alias(f"{var}_wdw[-(5day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((5*pl.col(f"{var}_wdw[-(5day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(6day):-(6day-1hour)]"))/6).alias(f"{var}_wdw[-(6day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((6*pl.col(f"{var}_wdw[-(6day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]"))/7).alias(f"{var}_wdw[-(1week):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        *(
            ((pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]") + pl.col(f"{var}_wdw[-(2week):-(2week-1hour)]"))/2).alias(f"{var}_wdw[-(2week):-(1week-1hour):1week]")
            for var in measured_values
        ),
        *(
            ((pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]") + pl.col(f"{var}_wdw[-(2week):-(2week-1hour)]") + pl.col(f"{var}_wdw[-(3week):-(3week-1hour)]"))/3).alias(f"{var}_wdw[-(3week):-(1week-1hour):1week]")
            for var in measured_values
        )
    )
)
df3_zscored.write_ipc("Z-score normalised, with lags.parquet")

In [17]:
df3_zscored.describe()

statistic,DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,CO(GT)_wdw[-(1hour):(0hour)],PT08.S1(CO)_wdw[-(1hour):(0hour)],NMHC(GT)_wdw[-(1hour):(0hour)],C6H6(GT)_wdw[-(1hour):(0hour)],PT08.S2(NMHC)_wdw[-(1hour):(0hour)],NOx(GT)_wdw[-(1hour):(0hour)],PT08.S3(NOx)_wdw[-(1hour):(0hour)],NO2(GT)_wdw[-(1hour):(0hour)],PT08.S4(NO2)_wdw[-(1hour):(0hour)],PT08.S5(O3)_wdw[-(1hour):(0hour)],T_wdw[-(1hour):(0hour)],RH_wdw[-(1hour):(0hour)],AH_wdw[-(1hour):(0hour)],CO(GT)_wdw[-(2hour):-(1hour)],PT08.S1(CO)_wdw[-(2hour):-(1hour)],NMHC(GT)_wdw[-(2hour):-(1hour)],C6H6(GT)_wdw[-(2hour):-(1hour)],PT08.S2(NMHC)_wdw[-(2hour):-(1hour)],NOx(GT)_wdw[-(2hour):-(1hour)],PT08.S3(NOx)_wdw[-(2hour):-(1hour)],NO2(GT)_wdw[-(2hour):-(1hour)],PT08.S4(NO2)_wdw[-(2hour):-(1hour)],PT08.S5(O3)_wdw[-(2hour):-(1hour)],T_wdw[-(2hour):-(1hour)],RH_wdw[-(2hour):-(1hour)],AH_wdw[-(2hour):-(1hour)],CO(GT)_wdw[-(3hour):-(2hour)],PT08.S1(CO)_wdw[-(3hour):-(2hour)],NMHC(GT)_wdw[-(3hour):-(2hour)],C6H6(GT)_wdw[-(3hour):-(2hour)],PT08.S2(NMHC)_wdw[-(3hour):-(2hour)],NOx(GT)_wdw[-(3hour):-(2hour)],PT08.S3(NOx)_wdw[-(3hour):-(2hour)],NO2(GT)_wdw[-(3hour):-(2hour)],PT08.S4(NO2)_wdw[-(3hour):-(2hour)],PT08.S5(O3)_wdw[-(3hour):-(2hour)],T_wdw[-(3hour):-(2hour)],RH_wdw[-(3hour):-(2hour)],AH_wdw[-(3hour):-(2hour)],CO(GT)_wdw[-(2hour):(0hour)],PT08.S1(CO)_wdw[-(2hour):(0hour)],NMHC(GT)_wdw[-(2hour):(0hour)],C6H6(GT)_wdw[-(2hour):(0hour)],PT08.S2(NMHC)_wdw[-(2hour):(0hour)],NOx(GT)_wdw[-(2hour):(0hour)],PT08.S3(NOx)_wdw[-(2hour):(0hour)],NO2(GT)_wdw[-(2hour):(0hour)],PT08.S4(NO2)_wdw[-(2hour):(0hour)],PT08.S5(O3)_wdw[-(2hour):(0hour)],T_wdw[-(2hour):(0hour)],RH_wdw[-(2hour):(0hour)],AH_wdw[-(2hour):(0hour)],CO(GT)_wdw[-(3hour):(0hour)],PT08.S1(CO)_wdw[-(3hour):(0hour)],NMHC(GT)_wdw[-(3hour):(0hour)],C6H6(GT)_wdw[-(3hour):(0hour)],PT08.S2(NMHC)_wdw[-(3hour):(0hour)],NOx(GT)_wdw[-(3hour):(0hour)],PT08.S3(NOx)_wdw[-(3hour):(0hour)],NO2(GT)_wdw[-(3hour):(0hour)],PT08.S4(NO2)_wdw[-(3hour):(0hour)],PT08.S5(O3)_wdw[-(3hour):(0hour)],T_wdw[-(3hour):(0hour)],RH_wdw[-(3hour):(0hour)],AH_wdw[-(3hour):(0hour)],CO(GT)_wdw[-(1week):(0hour)],PT08.S1(CO)_wdw[-(1week):(0hour)],NMHC(GT)_wdw[-(1week):(0hour)],C6H6(GT)_wdw[-(1week):(0hour)],PT08.S2(NMHC)_wdw[-(1week):(0hour)],NOx(GT)_wdw[-(1week):(0hour)],PT08.S3(NOx)_wdw[-(1week):(0hour)],NO2(GT)_wdw[-(1week):(0hour)],PT08.S4(NO2)_wdw[-(1week):(0hour)],PT08.S5(O3)_wdw[-(1week):(0hour)],T_wdw[-(1week):(0hour)],RH_wdw[-(1week):(0hour)],AH_wdw[-(1week):(0hour)],CO(GT)_wdw[-(1day):-(1day-1hour)],PT08.S1(CO)_wdw[-(1day):-(1day-1hour)],NMHC(GT)_wdw[-(1day):-(1day-1hour)],C6H6(GT)_wdw[-(1day):-(1day-1hour)],PT08.S2(NMHC)_wdw[-(1day):-(1day-1hour)],NOx(GT)_wdw[-(1day):-(1day-1hour)],PT08.S3(NOx)_wdw[-(1day):-(1day-1hour)],NO2(GT)_wdw[-(1day):-(1day-1hour)],PT08.S4(NO2)_wdw[-(1day):-(1day-1hour)],PT08.S5(O3)_wdw[-(1day):-(1day-1hour)],T_wdw[-(1day):-(1day-1hour)],RH_wdw[-(1day):-(1day-1hour)],AH_wdw[-(1day):-(1day-1hour)],CO(GT)_wdw[-(2day):-(2day-1hour)],PT08.S1(CO)_wdw[-(2day):-(2day-1hour)],NMHC(GT)_wdw[-(2day):-(2day-1hour)],C6H6(GT)_wdw[-(2day):-(2day-1hour)],PT08.S2(NMHC)_wdw[-(2day):-(2day-1hour)],NOx(GT)_wdw[-(2day):-(2day-1hour)],PT08.S3(NOx)_wdw[-(2day):-(2day-1hour)],NO2(GT)_wdw[-(2day):-(2day-1hour)],PT08.S4(NO2)_wdw[-(2day):-(2day-1hour)],PT08.S5(O3)_wdw[-(2day):-(2day-1hour)],T_wdw[-(2day):-(2day-1hour)],RH_wdw[-(2day):-(2day-1hour)],AH_wdw[-(2day):-(2day-1hour)],CO(GT)_wdw[-(3day):-(3day-1hour)],PT08.S1(CO)_wdw[-(3day):-(3day-1hour)],NMHC(GT)_wdw[-(3day):-(3day-1hour)],C6H6(GT)_wdw[-(3day):-(3day-1hour)],PT08.S2(NMHC)_wdw[-(3day):-(3day-1hour)],NOx(GT)_wdw[-(3day):-(3day-1hour)],PT08.S3(NOx)_wdw[-(3day):-(3day-1hour)],NO2(GT)_wdw[-(3day):-(3day-1hour)],PT08.S4(NO2)_wdw[-(3day):-(3day-1hour)],PT08

In [18]:
df3_zscored.head()

DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,CO(GT)_wdw[-(1hour):(0hour)],PT08.S1(CO)_wdw[-(1hour):(0hour)],NMHC(GT)_wdw[-(1hour):(0hour)],C6H6(GT)_wdw[-(1hour):(0hour)],PT08.S2(NMHC)_wdw[-(1hour):(0hour)],NOx(GT)_wdw[-(1hour):(0hour)],PT08.S3(NOx)_wdw[-(1hour):(0hour)],NO2(GT)_wdw[-(1hour):(0hour)],PT08.S4(NO2)_wdw[-(1hour):(0hour)],PT08.S5(O3)_wdw[-(1hour):(0hour)],T_wdw[-(1hour):(0hour)],RH_wdw[-(1hour):(0hour)],AH_wdw[-(1hour):(0hour)],CO(GT)_wdw[-(2hour):-(1hour)],PT08.S1(CO)_wdw[-(2hour):-(1hour)],NMHC(GT)_wdw[-(2hour):-(1hour)],C6H6(GT)_wdw[-(2hour):-(1hour)],PT08.S2(NMHC)_wdw[-(2hour):-(1hour)],NOx(GT)_wdw[-(2hour):-(1hour)],PT08.S3(NOx)_wdw[-(2hour):-(1hour)],NO2(GT)_wdw[-(2hour):-(1hour)],PT08.S4(NO2)_wdw[-(2hour):-(1hour)],PT08.S5(O3)_wdw[-(2hour):-(1hour)],T_wdw[-(2hour):-(1hour)],RH_wdw[-(2hour):-(1hour)],AH_wdw[-(2hour):-(1hour)],CO(GT)_wdw[-(3hour):-(2hour)],PT08.S1(CO)_wdw[-(3hour):-(2hour)],NMHC(GT)_wdw[-(3hour):-(2hour)],C6H6(GT)_wdw[-(3hour):-(2hour)],PT08.S2(NMHC)_wdw[-(3hour):-(2hour)],NOx(GT)_wdw[-(3hour):-(2hour)],PT08.S3(NOx)_wdw[-(3hour):-(2hour)],NO2(GT)_wdw[-(3hour):-(2hour)],PT08.S4(NO2)_wdw[-(3hour):-(2hour)],PT08.S5(O3)_wdw[-(3hour):-(2hour)],T_wdw[-(3hour):-(2hour)],RH_wdw[-(3hour):-(2hour)],AH_wdw[-(3hour):-(2hour)],CO(GT)_wdw[-(2hour):(0hour)],PT08.S1(CO)_wdw[-(2hour):(0hour)],NMHC(GT)_wdw[-(2hour):(0hour)],C6H6(GT)_wdw[-(2hour):(0hour)],PT08.S2(NMHC)_wdw[-(2hour):(0hour)],NOx(GT)_wdw[-(2hour):(0hour)],PT08.S3(NOx)_wdw[-(2hour):(0hour)],NO2(GT)_wdw[-(2hour):(0hour)],PT08.S4(NO2)_wdw[-(2hour):(0hour)],PT08.S5(O3)_wdw[-(2hour):(0hour)],T_wdw[-(2hour):(0hour)],RH_wdw[-(2hour):(0hour)],AH_wdw[-(2hour):(0hour)],CO(GT)_wdw[-(3hour):(0hour)],PT08.S1(CO)_wdw[-(3hour):(0hour)],NMHC(GT)_wdw[-(3hour):(0hour)],C6H6(GT)_wdw[-(3hour):(0hour)],PT08.S2(NMHC)_wdw[-(3hour):(0hour)],NOx(GT)_wdw[-(3hour):(0hour)],PT08.S3(NOx)_wdw[-(3hour):(0hour)],NO2(GT)_wdw[-(3hour):(0hour)],PT08.S4(NO2)_wdw[-(3hour):(0hour)],PT08.S5(O3)_wdw[-(3hour):(0hour)],T_wdw[-(3hour):(0hour)],RH_wdw[-(3hour):(0hour)],AH_wdw[-(3hour):(0hour)],CO(GT)_wdw[-(1week):(0hour)],PT08.S1(CO)_wdw[-(1week):(0hour)],NMHC(GT)_wdw[-(1week):(0hour)],C6H6(GT)_wdw[-(1week):(0hour)],PT08.S2(NMHC)_wdw[-(1week):(0hour)],NOx(GT)_wdw[-(1week):(0hour)],PT08.S3(NOx)_wdw[-(1week):(0hour)],NO2(GT)_wdw[-(1week):(0hour)],PT08.S4(NO2)_wdw[-(1week):(0hour)],PT08.S5(O3)_wdw[-(1week):(0hour)],T_wdw[-(1week):(0hour)],RH_wdw[-(1week):(0hour)],AH_wdw[-(1week):(0hour)],CO(GT)_wdw[-(1day):-(1day-1hour)],PT08.S1(CO)_wdw[-(1day):-(1day-1hour)],NMHC(GT)_wdw[-(1day):-(1day-1hour)],C6H6(GT)_wdw[-(1day):-(1day-1hour)],PT08.S2(NMHC)_wdw[-(1day):-(1day-1hour)],NOx(GT)_wdw[-(1day):-(1day-1hour)],PT08.S3(NOx)_wdw[-(1day):-(1day-1hour)],NO2(GT)_wdw[-(1day):-(1day-1hour)],PT08.S4(NO2)_wdw[-(1day):-(1day-1hour)],PT08.S5(O3)_wdw[-(1day):-(1day-1hour)],T_wdw[-(1day):-(1day-1hour)],RH_wdw[-(1day):-(1day-1hour)],AH_wdw[-(1day):-(1day-1hour)],CO(GT)_wdw[-(2day):-(2day-1hour)],PT08.S1(CO)_wdw[-(2day):-(2day-1hour)],NMHC(GT)_wdw[-(2day):-(2day-1hour)],C6H6(GT)_wdw[-(2day):-(2day-1hour)],PT08.S2(NMHC)_wdw[-(2day):-(2day-1hour)],NOx(GT)_wdw[-(2day):-(2day-1hour)],PT08.S3(NOx)_wdw[-(2day):-(2day-1hour)],NO2(GT)_wdw[-(2day):-(2day-1hour)],PT08.S4(NO2)_wdw[-(2day):-(2day-1hour)],PT08.S5(O3)_wdw[-(2day):-(2day-1hour)],T_wdw[-(2day):-(2day-1hour)],RH_wdw[-(2day):-(2day-1hour)],AH_wdw[-(2day):-(2day-1hour)],CO(GT)_wdw[-(3day):-(3day-1hour)],PT08.S1(CO)_wdw[-(3day):-(3day-1hour)],NMHC(GT)_wdw[-(3day):-(3day-1hour)],C6H6(GT)_wdw[-(3day):-(3day-1hour)],PT08.S2(NMHC)_wdw[-(3day):-(3day-1hour)],NOx(GT)_wdw[-(3day):-(3day-1hour)],PT08.S3(NOx)_wdw[-(3day):-(3day-1hour)],NO2(GT)_wdw[-(3day):-(3day-1hour)],PT08.S4(NO2)_wdw[-(3day):-(3day-1hour)],PT08.S5(O3)_wd

In [19]:
df3_zscored.tail()

DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,CO(GT)_wdw[-(1hour):(0hour)],PT08.S1(CO)_wdw[-(1hour):(0hour)],NMHC(GT)_wdw[-(1hour):(0hour)],C6H6(GT)_wdw[-(1hour):(0hour)],PT08.S2(NMHC)_wdw[-(1hour):(0hour)],NOx(GT)_wdw[-(1hour):(0hour)],PT08.S3(NOx)_wdw[-(1hour):(0hour)],NO2(GT)_wdw[-(1hour):(0hour)],PT08.S4(NO2)_wdw[-(1hour):(0hour)],PT08.S5(O3)_wdw[-(1hour):(0hour)],T_wdw[-(1hour):(0hour)],RH_wdw[-(1hour):(0hour)],AH_wdw[-(1hour):(0hour)],CO(GT)_wdw[-(2hour):-(1hour)],PT08.S1(CO)_wdw[-(2hour):-(1hour)],NMHC(GT)_wdw[-(2hour):-(1hour)],C6H6(GT)_wdw[-(2hour):-(1hour)],PT08.S2(NMHC)_wdw[-(2hour):-(1hour)],NOx(GT)_wdw[-(2hour):-(1hour)],PT08.S3(NOx)_wdw[-(2hour):-(1hour)],NO2(GT)_wdw[-(2hour):-(1hour)],PT08.S4(NO2)_wdw[-(2hour):-(1hour)],PT08.S5(O3)_wdw[-(2hour):-(1hour)],T_wdw[-(2hour):-(1hour)],RH_wdw[-(2hour):-(1hour)],AH_wdw[-(2hour):-(1hour)],CO(GT)_wdw[-(3hour):-(2hour)],PT08.S1(CO)_wdw[-(3hour):-(2hour)],NMHC(GT)_wdw[-(3hour):-(2hour)],C6H6(GT)_wdw[-(3hour):-(2hour)],PT08.S2(NMHC)_wdw[-(3hour):-(2hour)],NOx(GT)_wdw[-(3hour):-(2hour)],PT08.S3(NOx)_wdw[-(3hour):-(2hour)],NO2(GT)_wdw[-(3hour):-(2hour)],PT08.S4(NO2)_wdw[-(3hour):-(2hour)],PT08.S5(O3)_wdw[-(3hour):-(2hour)],T_wdw[-(3hour):-(2hour)],RH_wdw[-(3hour):-(2hour)],AH_wdw[-(3hour):-(2hour)],CO(GT)_wdw[-(2hour):(0hour)],PT08.S1(CO)_wdw[-(2hour):(0hour)],NMHC(GT)_wdw[-(2hour):(0hour)],C6H6(GT)_wdw[-(2hour):(0hour)],PT08.S2(NMHC)_wdw[-(2hour):(0hour)],NOx(GT)_wdw[-(2hour):(0hour)],PT08.S3(NOx)_wdw[-(2hour):(0hour)],NO2(GT)_wdw[-(2hour):(0hour)],PT08.S4(NO2)_wdw[-(2hour):(0hour)],PT08.S5(O3)_wdw[-(2hour):(0hour)],T_wdw[-(2hour):(0hour)],RH_wdw[-(2hour):(0hour)],AH_wdw[-(2hour):(0hour)],CO(GT)_wdw[-(3hour):(0hour)],PT08.S1(CO)_wdw[-(3hour):(0hour)],NMHC(GT)_wdw[-(3hour):(0hour)],C6H6(GT)_wdw[-(3hour):(0hour)],PT08.S2(NMHC)_wdw[-(3hour):(0hour)],NOx(GT)_wdw[-(3hour):(0hour)],PT08.S3(NOx)_wdw[-(3hour):(0hour)],NO2(GT)_wdw[-(3hour):(0hour)],PT08.S4(NO2)_wdw[-(3hour):(0hour)],PT08.S5(O3)_wdw[-(3hour):(0hour)],T_wdw[-(3hour):(0hour)],RH_wdw[-(3hour):(0hour)],AH_wdw[-(3hour):(0hour)],CO(GT)_wdw[-(1week):(0hour)],PT08.S1(CO)_wdw[-(1week):(0hour)],NMHC(GT)_wdw[-(1week):(0hour)],C6H6(GT)_wdw[-(1week):(0hour)],PT08.S2(NMHC)_wdw[-(1week):(0hour)],NOx(GT)_wdw[-(1week):(0hour)],PT08.S3(NOx)_wdw[-(1week):(0hour)],NO2(GT)_wdw[-(1week):(0hour)],PT08.S4(NO2)_wdw[-(1week):(0hour)],PT08.S5(O3)_wdw[-(1week):(0hour)],T_wdw[-(1week):(0hour)],RH_wdw[-(1week):(0hour)],AH_wdw[-(1week):(0hour)],CO(GT)_wdw[-(1day):-(1day-1hour)],PT08.S1(CO)_wdw[-(1day):-(1day-1hour)],NMHC(GT)_wdw[-(1day):-(1day-1hour)],C6H6(GT)_wdw[-(1day):-(1day-1hour)],PT08.S2(NMHC)_wdw[-(1day):-(1day-1hour)],NOx(GT)_wdw[-(1day):-(1day-1hour)],PT08.S3(NOx)_wdw[-(1day):-(1day-1hour)],NO2(GT)_wdw[-(1day):-(1day-1hour)],PT08.S4(NO2)_wdw[-(1day):-(1day-1hour)],PT08.S5(O3)_wdw[-(1day):-(1day-1hour)],T_wdw[-(1day):-(1day-1hour)],RH_wdw[-(1day):-(1day-1hour)],AH_wdw[-(1day):-(1day-1hour)],CO(GT)_wdw[-(2day):-(2day-1hour)],PT08.S1(CO)_wdw[-(2day):-(2day-1hour)],NMHC(GT)_wdw[-(2day):-(2day-1hour)],C6H6(GT)_wdw[-(2day):-(2day-1hour)],PT08.S2(NMHC)_wdw[-(2day):-(2day-1hour)],NOx(GT)_wdw[-(2day):-(2day-1hour)],PT08.S3(NOx)_wdw[-(2day):-(2day-1hour)],NO2(GT)_wdw[-(2day):-(2day-1hour)],PT08.S4(NO2)_wdw[-(2day):-(2day-1hour)],PT08.S5(O3)_wdw[-(2day):-(2day-1hour)],T_wdw[-(2day):-(2day-1hour)],RH_wdw[-(2day):-(2day-1hour)],AH_wdw[-(2day):-(2day-1hour)],CO(GT)_wdw[-(3day):-(3day-1hour)],PT08.S1(CO)_wdw[-(3day):-(3day-1hour)],NMHC(GT)_wdw[-(3day):-(3day-1hour)],C6H6(GT)_wdw[-(3day):-(3day-1hour)],PT08.S2(NMHC)_wdw[-(3day):-(3day-1hour)],NOx(GT)_wdw[-(3day):-(3day-1hour)],PT08.S3(NOx)_wdw[-(3day):-(3day-1hour)],NO2(GT)_wdw[-(3day):-(3day-1hour)],PT08.S4(NO2)_wdw[-(3day):-(3day-1hour)],PT08.S5(O3)_wd

In [20]:
df3_01_normed_lags = (
    df3_01_normed.select(
        *time_vars,
        *measured_values,
        *(
            (pl.col(var).rolling_sum(window_size=size_before_excl+1) - pl.col(var).rolling_sum(window_size=exclusion+1)).alias(f"{var}_wdw[{fmt_wdw_num(size_before_excl)}:{fmt_wdw_num(exclusion)}]") / (size_before_excl - exclusion)
            for size_before_excl, exclusion in windows
            for var in measured_values
        )
    )
    .with_columns(
        ((pl.col(f"{var}_wdw[-(1day):-(1day-1hour)]")        + pl.col(f"{var}_wdw[-(2day):-(2day-1hour)]"))/2).alias(f"{var}_wdw[-(2day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((2*pl.col(f"{var}_wdw[-(2day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(3day):-(3day-1hour)]"))/3).alias(f"{var}_wdw[-(3day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((3*pl.col(f"{var}_wdw[-(3day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(4day):-(4day-1hour)]"))/4).alias(f"{var}_wdw[-(4day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((4*pl.col(f"{var}_wdw[-(4day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(5day):-(5day-1hour)]"))/5).alias(f"{var}_wdw[-(5day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((5*pl.col(f"{var}_wdw[-(5day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(6day):-(6day-1hour)]"))/6).alias(f"{var}_wdw[-(6day):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        ((6*pl.col(f"{var}_wdw[-(6day):-(1day-1hour):1day]") + pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]"))/7).alias(f"{var}_wdw[-(1week):-(1day-1hour):1day]")
        for var in measured_values
    )
    .with_columns(
        *(
            ((pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]") + pl.col(f"{var}_wdw[-(2week):-(2week-1hour)]"))/2).alias(f"{var}_wdw[-(2week):-(1week-1hour):1week]")
            for var in measured_values
        ),
        *(
            ((pl.col(f"{var}_wdw[-(1week):-(1week-1hour)]") + pl.col(f"{var}_wdw[-(2week):-(2week-1hour)]") + pl.col(f"{var}_wdw[-(3week):-(3week-1hour)]"))/3).alias(f"{var}_wdw[-(3week):-(1week-1hour):1week]")
            for var in measured_values
        )
    )
)
# df3_zscored.write_ipc("Z-score normalised, with lags.parquet")
df3_01_normed_lags.write_ipc("min-max scaled, with lags.parquet")
df3_01_normed_lags.tail()

DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,CO(GT)_wdw[-(1hour):(0hour)],PT08.S1(CO)_wdw[-(1hour):(0hour)],NMHC(GT)_wdw[-(1hour):(0hour)],C6H6(GT)_wdw[-(1hour):(0hour)],PT08.S2(NMHC)_wdw[-(1hour):(0hour)],NOx(GT)_wdw[-(1hour):(0hour)],PT08.S3(NOx)_wdw[-(1hour):(0hour)],NO2(GT)_wdw[-(1hour):(0hour)],PT08.S4(NO2)_wdw[-(1hour):(0hour)],PT08.S5(O3)_wdw[-(1hour):(0hour)],T_wdw[-(1hour):(0hour)],RH_wdw[-(1hour):(0hour)],AH_wdw[-(1hour):(0hour)],CO(GT)_wdw[-(2hour):-(1hour)],PT08.S1(CO)_wdw[-(2hour):-(1hour)],NMHC(GT)_wdw[-(2hour):-(1hour)],C6H6(GT)_wdw[-(2hour):-(1hour)],PT08.S2(NMHC)_wdw[-(2hour):-(1hour)],NOx(GT)_wdw[-(2hour):-(1hour)],PT08.S3(NOx)_wdw[-(2hour):-(1hour)],NO2(GT)_wdw[-(2hour):-(1hour)],PT08.S4(NO2)_wdw[-(2hour):-(1hour)],PT08.S5(O3)_wdw[-(2hour):-(1hour)],T_wdw[-(2hour):-(1hour)],RH_wdw[-(2hour):-(1hour)],AH_wdw[-(2hour):-(1hour)],CO(GT)_wdw[-(3hour):-(2hour)],PT08.S1(CO)_wdw[-(3hour):-(2hour)],NMHC(GT)_wdw[-(3hour):-(2hour)],C6H6(GT)_wdw[-(3hour):-(2hour)],PT08.S2(NMHC)_wdw[-(3hour):-(2hour)],NOx(GT)_wdw[-(3hour):-(2hour)],PT08.S3(NOx)_wdw[-(3hour):-(2hour)],NO2(GT)_wdw[-(3hour):-(2hour)],PT08.S4(NO2)_wdw[-(3hour):-(2hour)],PT08.S5(O3)_wdw[-(3hour):-(2hour)],T_wdw[-(3hour):-(2hour)],RH_wdw[-(3hour):-(2hour)],AH_wdw[-(3hour):-(2hour)],CO(GT)_wdw[-(2hour):(0hour)],PT08.S1(CO)_wdw[-(2hour):(0hour)],NMHC(GT)_wdw[-(2hour):(0hour)],C6H6(GT)_wdw[-(2hour):(0hour)],PT08.S2(NMHC)_wdw[-(2hour):(0hour)],NOx(GT)_wdw[-(2hour):(0hour)],PT08.S3(NOx)_wdw[-(2hour):(0hour)],NO2(GT)_wdw[-(2hour):(0hour)],PT08.S4(NO2)_wdw[-(2hour):(0hour)],PT08.S5(O3)_wdw[-(2hour):(0hour)],T_wdw[-(2hour):(0hour)],RH_wdw[-(2hour):(0hour)],AH_wdw[-(2hour):(0hour)],CO(GT)_wdw[-(3hour):(0hour)],PT08.S1(CO)_wdw[-(3hour):(0hour)],NMHC(GT)_wdw[-(3hour):(0hour)],C6H6(GT)_wdw[-(3hour):(0hour)],PT08.S2(NMHC)_wdw[-(3hour):(0hour)],NOx(GT)_wdw[-(3hour):(0hour)],PT08.S3(NOx)_wdw[-(3hour):(0hour)],NO2(GT)_wdw[-(3hour):(0hour)],PT08.S4(NO2)_wdw[-(3hour):(0hour)],PT08.S5(O3)_wdw[-(3hour):(0hour)],T_wdw[-(3hour):(0hour)],RH_wdw[-(3hour):(0hour)],AH_wdw[-(3hour):(0hour)],CO(GT)_wdw[-(1week):(0hour)],PT08.S1(CO)_wdw[-(1week):(0hour)],NMHC(GT)_wdw[-(1week):(0hour)],C6H6(GT)_wdw[-(1week):(0hour)],PT08.S2(NMHC)_wdw[-(1week):(0hour)],NOx(GT)_wdw[-(1week):(0hour)],PT08.S3(NOx)_wdw[-(1week):(0hour)],NO2(GT)_wdw[-(1week):(0hour)],PT08.S4(NO2)_wdw[-(1week):(0hour)],PT08.S5(O3)_wdw[-(1week):(0hour)],T_wdw[-(1week):(0hour)],RH_wdw[-(1week):(0hour)],AH_wdw[-(1week):(0hour)],CO(GT)_wdw[-(1day):-(1day-1hour)],PT08.S1(CO)_wdw[-(1day):-(1day-1hour)],NMHC(GT)_wdw[-(1day):-(1day-1hour)],C6H6(GT)_wdw[-(1day):-(1day-1hour)],PT08.S2(NMHC)_wdw[-(1day):-(1day-1hour)],NOx(GT)_wdw[-(1day):-(1day-1hour)],PT08.S3(NOx)_wdw[-(1day):-(1day-1hour)],NO2(GT)_wdw[-(1day):-(1day-1hour)],PT08.S4(NO2)_wdw[-(1day):-(1day-1hour)],PT08.S5(O3)_wdw[-(1day):-(1day-1hour)],T_wdw[-(1day):-(1day-1hour)],RH_wdw[-(1day):-(1day-1hour)],AH_wdw[-(1day):-(1day-1hour)],CO(GT)_wdw[-(2day):-(2day-1hour)],PT08.S1(CO)_wdw[-(2day):-(2day-1hour)],NMHC(GT)_wdw[-(2day):-(2day-1hour)],C6H6(GT)_wdw[-(2day):-(2day-1hour)],PT08.S2(NMHC)_wdw[-(2day):-(2day-1hour)],NOx(GT)_wdw[-(2day):-(2day-1hour)],PT08.S3(NOx)_wdw[-(2day):-(2day-1hour)],NO2(GT)_wdw[-(2day):-(2day-1hour)],PT08.S4(NO2)_wdw[-(2day):-(2day-1hour)],PT08.S5(O3)_wdw[-(2day):-(2day-1hour)],T_wdw[-(2day):-(2day-1hour)],RH_wdw[-(2day):-(2day-1hour)],AH_wdw[-(2day):-(2day-1hour)],CO(GT)_wdw[-(3day):-(3day-1hour)],PT08.S1(CO)_wdw[-(3day):-(3day-1hour)],NMHC(GT)_wdw[-(3day):-(3day-1hour)],C6H6(GT)_wdw[-(3day):-(3day-1hour)],PT08.S2(NMHC)_wdw[-(3day):-(3day-1hour)],NOx(GT)_wdw[-(3day):-(3day-1hour)],PT08.S3(NOx)_wdw[-(3day):-(3day-1hour)],NO2(GT)_wdw[-(3day):-(3day-1hour)],PT08.S4(NO2)_wdw[-(3day):-(3day-1hour)],PT08.S5(O3)_wd

In [21]:
df3_01_normed_lags.head()

DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,CO(GT)_wdw[-(1hour):(0hour)],PT08.S1(CO)_wdw[-(1hour):(0hour)],NMHC(GT)_wdw[-(1hour):(0hour)],C6H6(GT)_wdw[-(1hour):(0hour)],PT08.S2(NMHC)_wdw[-(1hour):(0hour)],NOx(GT)_wdw[-(1hour):(0hour)],PT08.S3(NOx)_wdw[-(1hour):(0hour)],NO2(GT)_wdw[-(1hour):(0hour)],PT08.S4(NO2)_wdw[-(1hour):(0hour)],PT08.S5(O3)_wdw[-(1hour):(0hour)],T_wdw[-(1hour):(0hour)],RH_wdw[-(1hour):(0hour)],AH_wdw[-(1hour):(0hour)],CO(GT)_wdw[-(2hour):-(1hour)],PT08.S1(CO)_wdw[-(2hour):-(1hour)],NMHC(GT)_wdw[-(2hour):-(1hour)],C6H6(GT)_wdw[-(2hour):-(1hour)],PT08.S2(NMHC)_wdw[-(2hour):-(1hour)],NOx(GT)_wdw[-(2hour):-(1hour)],PT08.S3(NOx)_wdw[-(2hour):-(1hour)],NO2(GT)_wdw[-(2hour):-(1hour)],PT08.S4(NO2)_wdw[-(2hour):-(1hour)],PT08.S5(O3)_wdw[-(2hour):-(1hour)],T_wdw[-(2hour):-(1hour)],RH_wdw[-(2hour):-(1hour)],AH_wdw[-(2hour):-(1hour)],CO(GT)_wdw[-(3hour):-(2hour)],PT08.S1(CO)_wdw[-(3hour):-(2hour)],NMHC(GT)_wdw[-(3hour):-(2hour)],C6H6(GT)_wdw[-(3hour):-(2hour)],PT08.S2(NMHC)_wdw[-(3hour):-(2hour)],NOx(GT)_wdw[-(3hour):-(2hour)],PT08.S3(NOx)_wdw[-(3hour):-(2hour)],NO2(GT)_wdw[-(3hour):-(2hour)],PT08.S4(NO2)_wdw[-(3hour):-(2hour)],PT08.S5(O3)_wdw[-(3hour):-(2hour)],T_wdw[-(3hour):-(2hour)],RH_wdw[-(3hour):-(2hour)],AH_wdw[-(3hour):-(2hour)],CO(GT)_wdw[-(2hour):(0hour)],PT08.S1(CO)_wdw[-(2hour):(0hour)],NMHC(GT)_wdw[-(2hour):(0hour)],C6H6(GT)_wdw[-(2hour):(0hour)],PT08.S2(NMHC)_wdw[-(2hour):(0hour)],NOx(GT)_wdw[-(2hour):(0hour)],PT08.S3(NOx)_wdw[-(2hour):(0hour)],NO2(GT)_wdw[-(2hour):(0hour)],PT08.S4(NO2)_wdw[-(2hour):(0hour)],PT08.S5(O3)_wdw[-(2hour):(0hour)],T_wdw[-(2hour):(0hour)],RH_wdw[-(2hour):(0hour)],AH_wdw[-(2hour):(0hour)],CO(GT)_wdw[-(3hour):(0hour)],PT08.S1(CO)_wdw[-(3hour):(0hour)],NMHC(GT)_wdw[-(3hour):(0hour)],C6H6(GT)_wdw[-(3hour):(0hour)],PT08.S2(NMHC)_wdw[-(3hour):(0hour)],NOx(GT)_wdw[-(3hour):(0hour)],PT08.S3(NOx)_wdw[-(3hour):(0hour)],NO2(GT)_wdw[-(3hour):(0hour)],PT08.S4(NO2)_wdw[-(3hour):(0hour)],PT08.S5(O3)_wdw[-(3hour):(0hour)],T_wdw[-(3hour):(0hour)],RH_wdw[-(3hour):(0hour)],AH_wdw[-(3hour):(0hour)],CO(GT)_wdw[-(1week):(0hour)],PT08.S1(CO)_wdw[-(1week):(0hour)],NMHC(GT)_wdw[-(1week):(0hour)],C6H6(GT)_wdw[-(1week):(0hour)],PT08.S2(NMHC)_wdw[-(1week):(0hour)],NOx(GT)_wdw[-(1week):(0hour)],PT08.S3(NOx)_wdw[-(1week):(0hour)],NO2(GT)_wdw[-(1week):(0hour)],PT08.S4(NO2)_wdw[-(1week):(0hour)],PT08.S5(O3)_wdw[-(1week):(0hour)],T_wdw[-(1week):(0hour)],RH_wdw[-(1week):(0hour)],AH_wdw[-(1week):(0hour)],CO(GT)_wdw[-(1day):-(1day-1hour)],PT08.S1(CO)_wdw[-(1day):-(1day-1hour)],NMHC(GT)_wdw[-(1day):-(1day-1hour)],C6H6(GT)_wdw[-(1day):-(1day-1hour)],PT08.S2(NMHC)_wdw[-(1day):-(1day-1hour)],NOx(GT)_wdw[-(1day):-(1day-1hour)],PT08.S3(NOx)_wdw[-(1day):-(1day-1hour)],NO2(GT)_wdw[-(1day):-(1day-1hour)],PT08.S4(NO2)_wdw[-(1day):-(1day-1hour)],PT08.S5(O3)_wdw[-(1day):-(1day-1hour)],T_wdw[-(1day):-(1day-1hour)],RH_wdw[-(1day):-(1day-1hour)],AH_wdw[-(1day):-(1day-1hour)],CO(GT)_wdw[-(2day):-(2day-1hour)],PT08.S1(CO)_wdw[-(2day):-(2day-1hour)],NMHC(GT)_wdw[-(2day):-(2day-1hour)],C6H6(GT)_wdw[-(2day):-(2day-1hour)],PT08.S2(NMHC)_wdw[-(2day):-(2day-1hour)],NOx(GT)_wdw[-(2day):-(2day-1hour)],PT08.S3(NOx)_wdw[-(2day):-(2day-1hour)],NO2(GT)_wdw[-(2day):-(2day-1hour)],PT08.S4(NO2)_wdw[-(2day):-(2day-1hour)],PT08.S5(O3)_wdw[-(2day):-(2day-1hour)],T_wdw[-(2day):-(2day-1hour)],RH_wdw[-(2day):-(2day-1hour)],AH_wdw[-(2day):-(2day-1hour)],CO(GT)_wdw[-(3day):-(3day-1hour)],PT08.S1(CO)_wdw[-(3day):-(3day-1hour)],NMHC(GT)_wdw[-(3day):-(3day-1hour)],C6H6(GT)_wdw[-(3day):-(3day-1hour)],PT08.S2(NMHC)_wdw[-(3day):-(3day-1hour)],NOx(GT)_wdw[-(3day):-(3day-1hour)],PT08.S3(NOx)_wdw[-(3day):-(3day-1hour)],NO2(GT)_wdw[-(3day):-(3day-1hour)],PT08.S4(NO2)_wdw[-(3day):-(3day-1hour)],PT08.S5(O3)_wd

In [22]:
df3_01_normed_lags.describe()

statistic,DateTime,Date,Time,linear_dt,sin_time_of_year,cos_time_of_year,is_mon,is_tue,is_wed,is_thu,is_fri,is_sat,is_sun,is_weekday,sin_time_of_week,cos_time_of_week,sin_hour_of_day,cos_hour_of_day,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,CO(GT)_wdw[-(1hour):(0hour)],PT08.S1(CO)_wdw[-(1hour):(0hour)],NMHC(GT)_wdw[-(1hour):(0hour)],C6H6(GT)_wdw[-(1hour):(0hour)],PT08.S2(NMHC)_wdw[-(1hour):(0hour)],NOx(GT)_wdw[-(1hour):(0hour)],PT08.S3(NOx)_wdw[-(1hour):(0hour)],NO2(GT)_wdw[-(1hour):(0hour)],PT08.S4(NO2)_wdw[-(1hour):(0hour)],PT08.S5(O3)_wdw[-(1hour):(0hour)],T_wdw[-(1hour):(0hour)],RH_wdw[-(1hour):(0hour)],AH_wdw[-(1hour):(0hour)],CO(GT)_wdw[-(2hour):-(1hour)],PT08.S1(CO)_wdw[-(2hour):-(1hour)],NMHC(GT)_wdw[-(2hour):-(1hour)],C6H6(GT)_wdw[-(2hour):-(1hour)],PT08.S2(NMHC)_wdw[-(2hour):-(1hour)],NOx(GT)_wdw[-(2hour):-(1hour)],PT08.S3(NOx)_wdw[-(2hour):-(1hour)],NO2(GT)_wdw[-(2hour):-(1hour)],PT08.S4(NO2)_wdw[-(2hour):-(1hour)],PT08.S5(O3)_wdw[-(2hour):-(1hour)],T_wdw[-(2hour):-(1hour)],RH_wdw[-(2hour):-(1hour)],AH_wdw[-(2hour):-(1hour)],CO(GT)_wdw[-(3hour):-(2hour)],PT08.S1(CO)_wdw[-(3hour):-(2hour)],NMHC(GT)_wdw[-(3hour):-(2hour)],C6H6(GT)_wdw[-(3hour):-(2hour)],PT08.S2(NMHC)_wdw[-(3hour):-(2hour)],NOx(GT)_wdw[-(3hour):-(2hour)],PT08.S3(NOx)_wdw[-(3hour):-(2hour)],NO2(GT)_wdw[-(3hour):-(2hour)],PT08.S4(NO2)_wdw[-(3hour):-(2hour)],PT08.S5(O3)_wdw[-(3hour):-(2hour)],T_wdw[-(3hour):-(2hour)],RH_wdw[-(3hour):-(2hour)],AH_wdw[-(3hour):-(2hour)],CO(GT)_wdw[-(2hour):(0hour)],PT08.S1(CO)_wdw[-(2hour):(0hour)],NMHC(GT)_wdw[-(2hour):(0hour)],C6H6(GT)_wdw[-(2hour):(0hour)],PT08.S2(NMHC)_wdw[-(2hour):(0hour)],NOx(GT)_wdw[-(2hour):(0hour)],PT08.S3(NOx)_wdw[-(2hour):(0hour)],NO2(GT)_wdw[-(2hour):(0hour)],PT08.S4(NO2)_wdw[-(2hour):(0hour)],PT08.S5(O3)_wdw[-(2hour):(0hour)],T_wdw[-(2hour):(0hour)],RH_wdw[-(2hour):(0hour)],AH_wdw[-(2hour):(0hour)],CO(GT)_wdw[-(3hour):(0hour)],PT08.S1(CO)_wdw[-(3hour):(0hour)],NMHC(GT)_wdw[-(3hour):(0hour)],C6H6(GT)_wdw[-(3hour):(0hour)],PT08.S2(NMHC)_wdw[-(3hour):(0hour)],NOx(GT)_wdw[-(3hour):(0hour)],PT08.S3(NOx)_wdw[-(3hour):(0hour)],NO2(GT)_wdw[-(3hour):(0hour)],PT08.S4(NO2)_wdw[-(3hour):(0hour)],PT08.S5(O3)_wdw[-(3hour):(0hour)],T_wdw[-(3hour):(0hour)],RH_wdw[-(3hour):(0hour)],AH_wdw[-(3hour):(0hour)],CO(GT)_wdw[-(1week):(0hour)],PT08.S1(CO)_wdw[-(1week):(0hour)],NMHC(GT)_wdw[-(1week):(0hour)],C6H6(GT)_wdw[-(1week):(0hour)],PT08.S2(NMHC)_wdw[-(1week):(0hour)],NOx(GT)_wdw[-(1week):(0hour)],PT08.S3(NOx)_wdw[-(1week):(0hour)],NO2(GT)_wdw[-(1week):(0hour)],PT08.S4(NO2)_wdw[-(1week):(0hour)],PT08.S5(O3)_wdw[-(1week):(0hour)],T_wdw[-(1week):(0hour)],RH_wdw[-(1week):(0hour)],AH_wdw[-(1week):(0hour)],CO(GT)_wdw[-(1day):-(1day-1hour)],PT08.S1(CO)_wdw[-(1day):-(1day-1hour)],NMHC(GT)_wdw[-(1day):-(1day-1hour)],C6H6(GT)_wdw[-(1day):-(1day-1hour)],PT08.S2(NMHC)_wdw[-(1day):-(1day-1hour)],NOx(GT)_wdw[-(1day):-(1day-1hour)],PT08.S3(NOx)_wdw[-(1day):-(1day-1hour)],NO2(GT)_wdw[-(1day):-(1day-1hour)],PT08.S4(NO2)_wdw[-(1day):-(1day-1hour)],PT08.S5(O3)_wdw[-(1day):-(1day-1hour)],T_wdw[-(1day):-(1day-1hour)],RH_wdw[-(1day):-(1day-1hour)],AH_wdw[-(1day):-(1day-1hour)],CO(GT)_wdw[-(2day):-(2day-1hour)],PT08.S1(CO)_wdw[-(2day):-(2day-1hour)],NMHC(GT)_wdw[-(2day):-(2day-1hour)],C6H6(GT)_wdw[-(2day):-(2day-1hour)],PT08.S2(NMHC)_wdw[-(2day):-(2day-1hour)],NOx(GT)_wdw[-(2day):-(2day-1hour)],PT08.S3(NOx)_wdw[-(2day):-(2day-1hour)],NO2(GT)_wdw[-(2day):-(2day-1hour)],PT08.S4(NO2)_wdw[-(2day):-(2day-1hour)],PT08.S5(O3)_wdw[-(2day):-(2day-1hour)],T_wdw[-(2day):-(2day-1hour)],RH_wdw[-(2day):-(2day-1hour)],AH_wdw[-(2day):-(2day-1hour)],CO(GT)_wdw[-(3day):-(3day-1hour)],PT08.S1(CO)_wdw[-(3day):-(3day-1hour)],NMHC(GT)_wdw[-(3day):-(3day-1hour)],C6H6(GT)_wdw[-(3day):-(3day-1hour)],PT08.S2(NMHC)_wdw[-(3day):-(3day-1hour)],NOx(GT)_wdw[-(3day):-(3day-1hour)],PT08.S3(NOx)_wdw[-(3day):-(3day-1hour)],NO2(GT)_wdw[-(3day):-(3day-1hour)],PT08.S4(NO2)_wdw[-(3day):-(3day-1hour)],PT08